In [ ]:
import pandas as pd
import os
# from google.colab import drive
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import scipy
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from math import sqrt
from sklearn.model_selection import cross_val_score,cross_val_predict,KFold,train_test_split,StratifiedKFold
from sklearn.linear_model import LogisticRegression


import imblearn
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score, classification_report
import time

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/water_potability_csv.csv')

In [ ]:
data

,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity,Potability
0,NaN,204.890456,20791.31898,7.300212,368.516441,564.308654,10.379783,86.990970,2.963135,0
1,3.716080,129.422921,18630.05786,6.635246,NaN,592.885359,15.180013,56.329076,4.500656,0
2,8.099124,224.236259,19909.54173,9.275884,NaN,418.606213,16.868637,66.420093,3.055934,0
3,8.316766,214.373394,22018.41744,8.059332,356.886136,363.266516,18.436525,100.341674,4.628771,0
4,9.092223,181.101509,17978.98634,6.546600,310.135738,398.410813,11.558279,31.997993,4.075075,0
...,...,...,...,...,...,...,...,...,...,...
3271,4.668102,193.681736,47580.99160,7.166639,359.948574,526.424171,13.894419,66.687695,4.435821,1
3272,7.808856,193.553212,17329.80216,8.061362,NaN,392.449580,19.903225,NaN,2.798243,1
3273,9.419510,175.762646,33155.57822,7.350233,NaN,432.044783,11.039070,69.845400,3.298875,1
3274,5.126763,230.603758,11983.86938,6.303357,NaN,402.883113,11.168946,77.488213,4.708658,1


In [ ]:
data.isnull().sum()

ph                 491
Hardness             0
Solids               0
Chloramines          0
Sulfate            781
Conductivity         0
Organic_carbon       0
Trihalomethanes    162
Turbidity            0
Potability           0
dtype: int64

In [ ]:
# cek data kosong, jika ada yang kosong dihapus
data = data.dropna()
# cek data jumlah kosong
data.isnull().sum()

ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
Potability         0
dtype: int64

In [ ]:
print(data)

             ph    Hardness       Solids  Chloramines     Sulfate  \
3      8.316766  214.373394  22018.41744     8.059332  356.886136   
4      9.092223  181.101509  17978.98634     6.546600  310.135738   
5      5.584087  188.313324  28748.68774     7.544869  326.678363   
6     10.223862  248.071735  28749.71654     7.513408  393.663395   
7      8.635849  203.361523  13672.09176     4.563009  303.309771   
...         ...         ...          ...          ...         ...   
3267   8.989900  215.047358  15921.41202     6.297312  312.931021   
3268   6.702547  207.321086  17246.92035     7.708117  304.510230   
3269  11.491011   94.812545  37188.82602     9.263166  258.930600   
3270   6.069616  186.659040  26138.78019     7.747547  345.700257   
3271   4.668102  193.681736  47580.99160     7.166639  359.948574   

      Conductivity  Organic_carbon  Trihalomethanes  Turbidity  Potability  
3       363.266516       18.436525       100.341674   4.628771           0  
4       398.41081

In [ ]:
data.min()

ph                   0.227499
Hardness            73.492234
Solids             320.942611
Chloramines          1.390871
Sulfate            129.000000
Conductivity       201.619737
Organic_carbon       2.200000
Trihalomethanes      8.577013
Turbidity            1.450000
Potability           0.000000
dtype: float64

In [ ]:
data.max()

ph                    14.000000
Hardness             317.338124
Solids             56488.672410
Chloramines           13.127000
Sulfate              481.030642
Conductivity         753.342620
Organic_carbon        27.006707
Trihalomethanes      124.000000
Turbidity              6.494749
Potability             1.000000
dtype: float64

In [ ]:
X = data.drop(columns="Potability")
y = data.Potability

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(X)

X_scaled = scaler.transform(X)

In [ ]:
X_scaled

array([[0.58734916, 0.57774671, 0.38629788, ..., 0.65452157, 0.79502934,
        0.63011476],
       [0.64365393, 0.44130034, 0.31438058, ..., 0.37724796, 0.20291434,
        0.52035803],
       [0.38893354, 0.47087564, 0.50612238, ..., 0.24992171, 0.40148717,
        0.21997295],
       ...,
       [0.81782618, 0.08743355, 0.65638906, ..., 0.56326524, 0.28574454,
        0.5786739 ],
       [0.42418706, 0.4640915 , 0.45965606, ..., 0.39778032, 0.44915584,
        0.44000443],
       [0.32242529, 0.49289123, 0.84140928, ..., 0.47142165, 0.50345848,
        0.59186714]])

In [ ]:
y_array = np.array(y)

In [ ]:
print(y_array.count([1, 0]))

AttributeError: ignored

In [ ]:
# KFOLD CROSS VALIDATION
list_akurasi=[]
i = range(3,44,2)
akurasirendah = 1
akurasiterbaik = 0
kterbaik = 0
for k in i:
  knn = KNeighborsClassifier(n_neighbors=k)
  akurasi = cross_val_score(knn, X_scaled, y, cv=10)
  # mecari akurasi tertinggi
  list_akurasi.append(akurasi.mean())
  if akurasiterbaik < akurasi.mean():
    akurasiterbaik = akurasi.mean()
    kterbaik = k
  # mencari akurasi terendah
  list_akurasi.append(akurasi.mean())
  if akurasirendah > akurasi.mean():
    akurasirendah = akurasi.mean()
    kterendah = k
  print(k,akurasi.mean())
print('nilai paling tertinggi pada k :',kterbaik,max(list_akurasi))
print('nilai paling terendah pada k :',kterendah,min(list_akurasi))
  

3 0.606652381656076
5 0.6011772819072951
7 0.6041672823998818
9 0.617587803556475
11 0.6185877542978179
13 0.6190581744741638
15 0.6300083739717255
17 0.6300132998374465
19 0.6220457120338899
21 0.6295034727353332
23 0.6275232747155313
25 0.6260258115363775
27 0.6245431259543865
29 0.6260405891335402
31 0.6205679523176199
33 0.623555489877346
35 0.6210654647554308
37 0.6260332003349588
39 0.6240456135165755
41 0.6300083739717256
43 0.6250431013250579
nilai paling tertinggi pada k : 17 0.6300132998374465
nilai paling terendah pada k : 5 0.6011772819072951


In [ ]:
max(list_akurasi)

0.6300132998374465

In [ ]:
min(list_akurasi)

0.6011772819072951

In [ ]:
knn = KNeighborsClassifier(n_neighbors=kterbaik)

In [ ]:
testing = cross_val_predict(knn,X_scaled,y,cv=10)

In [ ]:
confusion_matrix(y_array,testing,labels=[0,1])

array([[1052,  148],
       [ 596,  215]])

In [ ]:
accuracy_score(y_array,testing)

0.6300348085529587

In [ ]:
precision_score(y_array,testing,labels=[0,1])

0.5922865013774105

In [ ]:
recall_score(y_array,testing,labels=[0,1])

0.2651048088779285

In [ ]:
f1_score(y_array,testing,labels=[0,1])

0.3662691652470187

In [ ]:
print(classification_report (y,testing))

              precision    recall  f1-score   support

           0       0.64      0.88      0.74      1200
           1       0.59      0.27      0.37       811

    accuracy                           0.63      2011
   macro avg       0.62      0.57      0.55      2011
weighted avg       0.62      0.63      0.59      2011

